In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/hw4

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/hw4


In [ ]:
!pip install torch
!pip install transformers
!pip install datasets

In [ ]:
import torch
from transformers import GPT2Tokenizer, AutoTokenizer, GPT2LMHeadModel
from datasets import load_dataset
from collections import defaultdict
import copy

## **Task 1: Byte pair encoding**

The goal is to calculate byte pair encodings for a given dataset. For a recap of the concept we refer to lecture 9, slides 22.ff and internet search.

### Task 1.1
First we want to make sure we create a suffieciently large corpus. As dataset you import the dataset "wikitext-2-raw-v1" from huggingface. Make sure to use the 500 first entries of the training dataset and subsequently filter out empty  entries. Also print the first entry of the corpus.

In [ ]:
# Load WikiText-2 dataset "wikitext-2-raw-v1"
wiki_dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
train_dataset = wiki_dataset["train"]
corpus = []
# YOUR CODE HERE
#iterate over the first 500 entries of the trainingset and filter out empty entries
for sent in train_dataset["text"][:500]:
  if sent not in ["", " "]:
    corpus.append(sent)

print(f"First Entry == {corpus[0]}")

  0%|          | 0/3 [00:00<?, ?it/s]

First Entry ==  = Valkyria Chronicles III = 



Note that the corpus should have a format like
corpus = [
    "This is the first sentence",
    "This is the second sentence",
    ..
]. For debugging during coding you can also consider working with a smaller corpus, e.g. only 5 sentences

### Task 1.2
Here, you loop through the corpus and count the word frequencies

In [ ]:
# Initialize a tokenizer from the transformers library
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Initialize a defaultdict to store word frequencies
word_freqs = defaultdict(int)
# this dictionary gets entries for word_freqs[word]

alphabet = []
vocab = [""]
splits = {}
vocab_size = 200
merges = {}

# We loop through to corups to calculate word frequencies
for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    # YOUR CODE HERE to count word frequencies
    for w in new_words:
      word_freqs[w] += 1

The following block creates the alphabet and initial vocabulary

In [ ]:
for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

print("alphabet",alphabet)
vocab = ["<|endoftext|>"] + alphabet.copy()
splits = {word: [c for c in word] for word in word_freqs.keys()}
splits_1000 = copy.deepcopy(splits)
splits_5000 = copy.deepcopy(splits)

alphabet ['!', '"', '$', '%', '&', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~', '¡', '¢', '£', '¤', '¥', '¦', '©', 'ª', '«', '®', '¯', '°', '±', '²', '³', '´', 'µ', '¹', '¼', '½', '¿', 'Â', 'Ã', 'Å', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ï', 'Ċ', 'Ġ', 'Ģ', 'ģ', 'Ĥ', 'ĥ', 'Ħ', 'ħ', 'Ĩ', 'Ī', 'ī', 'Ĭ', 'ĭ', 'Į', 'į', 'ı', 'Ĳ', 'ĳ', 'Ĵ', 'ĵ', 'Ķ', 'Ĺ', 'ĺ', 'Ļ', 'Ľ', 'Ł', 'ł', 'Ń']


### Task 1.3
Define a function to compute the frequency of each pair of characters

In [ ]:
# Define a function to compute the frequency of each pair of characters
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    # YOUR CODE HERE to compute the frequency of each pair of characters
    for k, v in splits.items():
      #v is a list of characters
      for i in range(len(v) - 1):
        pair = (v[i], v[i+1])
        pair_freqs[pair] += 1
    return pair_freqs

# Initialize pair_freqs by calling the compute_pair_freqs function
pair_freqs = compute_pair_freqs(splits)

Here we find the pair of characters that appears most frequently together

In [ ]:
for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print("\nMost frequent pair of characters : ", best_pair, " occurs ", max_freq, " times")

('Ġ', '='): 1
('Ġ', 'V'): 29
('V', 'a'): 10
('a', 'l'): 311
('l', 'k'): 9
('k', 'y'): 5

Most frequent pair of characters :  ('i', 'n')  occurs  582  times


### Task 1.4
Define a function to merge the most frequent pair of characters

In [ ]:
def help_fct(a, b, split):
  merged_list = []
  i = 0
  while i < len(split):
    if split[i] == a and i + 1 < len(split) and split[i + 1] == b:
      merged_list.append(a + b)
      i += 2
    else:
      merged_list.append(split[i])
      i += 1

  return merged_list

In [ ]:

# Define a function to merge the most frequent pair of characters
def merge_pair(a, b, splits):
    #splits_copy = copy.deepcopy(splits)
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue

        # YOUR CODE HERE to merge the most frequent pair of characters in each word
        splits[word] = help_fct(a=a, b=b, split=split)
    return splits

### Task 1.5

Iterate this process until you reach a predefined vocabulary size

In [ ]:
# YOUR CODE HERE to iterate this process until you reach a predefined vocabulary size
# Edited my this part of my code based on the following link: https://huggingface.co/learn/nlp-course/chapter6/5?fw=pt
vocab_size = 200
# initialize the merges variable with the most frequent character pair, because it won't be taken into consideration in the loop block.
merges = {("i", "n"): "in"}
vocab.append("in")
while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

In [ ]:
print(merges)

{('i', 'n'): 'in', ('e', 'r'): 'er', ('e', 'd'): 'ed', ('o', 'n'): 'on', ('e', 's'): 'es', ('e', 'n'): 'en', ('Ġ', 's'): 'Ġs', ('a', 't'): 'at', ('a', 'n'): 'an', ('a', 'l'): 'al', ('o', 'r'): 'or', ('a', 'r'): 'ar', ('in', 'g'): 'ing', ('Ġ', 'c'): 'Ġc', ('Ġ', 'p'): 'Ġp', ('r', 'e'): 're', ('t', 'i'): 'ti', ('i', 's'): 'is', ('Ġ', 'd'): 'Ġd', ('e', 'l'): 'el', ('Ġ', 'm'): 'Ġm', ('Ġ', 'f'): 'Ġf', ('i', 'c'): 'ic', ('i', 't'): 'it', ('Ġ', 't'): 'Ġt', ('o', 'u'): 'ou', ('l', 'e'): 'le', ('Ġ', 'a'): 'Ġa', ('r', 'o'): 'ro', ('Ġ', 'S'): 'ĠS', ('i', 'on'): 'ion', ('Ġ', 'b'): 'Ġb', ('en', 't'): 'ent', ('Ġ', 'C'): 'ĠC', ('Ġ', 're'): 'Ġre', ('Ġ', '1'): 'Ġ1', ('i', 'l'): 'il', ('o', 'm'): 'om', ('a', 's'): 'as', ('a', 'c'): 'ac', ('e', 'c'): 'ec', ('Ġ', 'w'): 'Ġw', ('Ġ', 'A'): 'ĠA', ('l', 'o'): 'lo', ('s', 't'): 'st', ('l', 'y'): 'ly', ('Ġ', 'e'): 'Ġe', ('Ġ', 'M'): 'ĠM', ('e', 't'): 'et', ('Ġ', 'in'): 'Ġin', ('Ġ', 'P'): 'ĠP', ('Ġ', 'h'): 'Ġh', ('u', 'r'): 'ur', ('er', 's'): 'ers', ('r', 'a'): 'ra

In [ ]:
print(vocab)

['<|endoftext|>', '!', '"', '$', '%', '&', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~', '¡', '¢', '£', '¤', '¥', '¦', '©', 'ª', '«', '®', '¯', '°', '±', '²', '³', '´', 'µ', '¹', '¼', '½', '¿', 'Â', 'Ã', 'Å', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ï', 'Ċ', 'Ġ', 'Ģ', 'ģ', 'Ĥ', 'ĥ', 'Ħ', 'ħ', 'Ĩ', 'Ī', 'ī', 'Ĭ', 'ĭ', 'Į', 'į', 'ı', 'Ĳ', 'ĳ', 'Ĵ', 'ĵ', 'Ķ', 'Ĺ', 'ĺ', 'Ļ', 'Ľ', 'Ł', 'ł', 'Ń', 'in', 'in', 'er', 'ed', 'on', 'es', 'en', 'Ġs', 'at', 'an', 'al', 'or', 'ar', 'ing', 'Ġc', 'Ġp', 're', 'ti', 'is', 'Ġd', 'el', 'Ġm', 'Ġf', 'ic', 'it', 'Ġt', 'ou', 'le', 'Ġa', 'ro', 'ĠS', 'ion', 'Ġb', 'ent', 'ĠC', 'Ġre', 'Ġ1', 'il', 'om', 'as', 'ac', 'ec', 'Ġw', 'ĠA',

In [ ]:
len(vocab)

200

In [ ]:
print(splits)

{'Ġ=': ['Ġ', '='], 'ĠValkyria': ['Ġ', 'V', 'al', 'k', 'y', 'r', 'i', 'a'], 'ĠChronicles': ['ĠC', 'h', 'r', 'on', 'ic', 'l', 'es'], 'ĠIII': ['Ġ', 'I', 'I', 'I'], 'ĠĊ': ['Ġ', 'Ċ'], 'ĠSenjÅį': ['ĠS', 'en', 'j', 'Å', 'į'], 'Ġno': ['Ġ', 'n', 'o'], 'Ġ3': ['Ġ', '3'], 'Ġ:': ['Ġ', ':'], 'ĠUnrecorded': ['Ġ', 'U', 'n', 're', 'c', 'or', 'd', 'ed'], 'Ġ(': ['Ġ', '('], 'ĠJapanese': ['Ġ', 'J', 'a', 'p', 'an', 'es', 'e'], 'ĠæĪ¦åł´ãģ®ãĥ´ãĤ¡ãĥ«ãĤŃãĥ¥ãĥªãĤ¢': ['Ġ', 'æ', 'Ī', '¦', 'å', 'ł', '´', 'ã', 'ģ', '®', 'ã', 'ĥ', '´', 'ã', 'Ĥ', '¡', 'ã', 'ĥ', '«', 'ã', 'Ĥ', 'Ń', 'ã', 'ĥ', '¥', 'ã', 'ĥ', 'ª', 'ã', 'Ĥ', '¢'], '3': ['3'], 'Ġ,': ['Ġ', ','], 'Ġlit': ['Ġ', 'l', 'it'], 'Ġ.': ['Ġ', '.'], 'Ġof': ['Ġ', 'o', 'f'], 'Ġthe': ['Ġt', 'h', 'e'], 'ĠBattlefield': ['Ġ', 'B', 'at', 't', 'le', 'f', 'i', 'el', 'd'], 'Ġ)': ['Ġ', ')'], 'Ġcommonly': ['Ġc', 'om', 'm', 'on', 'ly'], 'Ġreferred': ['Ġre', 'f', 'er', 'r', 'ed'], 'Ġto': ['Ġt', 'o'], 'Ġas': ['Ġa', 's'], 'Ġoutside': ['Ġ', 'ou', 't', 's', 'i', 'd', 'e'], 'ĠJapan': ['Ġ

We provide the tokenize function to you

In [ ]:
sentence_example = "Darmstadt holds the official title City of Science (German: Wissenschaftsstadt) as it is a major centre of scientific institutions, universities, and high-technology companies."
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    splits = [[l for l in word] for word in pre_tokenized_text]
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    return sum(splits, [])
print(tokenize(sentence_example))
print(f"Length with vocab size of 200 == {len(tokenize(sentence_example))}")

['D', 'ar', 'm', 'st', 'a', 'd', 't', 'Ġh', 'o', 'l', 'd', 's', 'Ġt', 'h', 'e', 'Ġ', 'o', 'f', 'f', 'ic', 'i', 'al', 'Ġ', 'ti', 't', 'le', 'ĠC', 'it', 'y', 'Ġ', 'o', 'f', 'ĠS', 'c', 'i', 'en', 'c', 'e', 'Ġ', '(', 'G', 'er', 'm', 'an', ':', 'Ġ', 'W', 'is', 's', 'en', 's', 'c', 'h', 'a', 'f', 't', 's', 'st', 'a', 'd', 't', ')', 'Ġa', 's', 'Ġ', 'it', 'Ġ', 'is', 'Ġa', 'Ġm', 'a', 'j', 'or', 'Ġc', 'ent', 're', 'Ġ', 'o', 'f', 'Ġs', 'c', 'i', 'en', 'ti', 'f', 'ic', 'Ġin', 's', 'ti', 't', 'u', 'ti', 'on', 's', ',', 'Ġ', 'u', 'n', 'i', 'v', 'ers', 'i', 'ti', 'es', ',', 'Ġ', 'an', 'd', 'Ġh', 'i', 'g', 'h', '-', 't', 'ec', 'h', 'n', 'o', 'lo', 'g', 'y', 'Ġc', 'om', 'p', 'an', 'i', 'es', '.']
Length with vocab size of 200 == 128


### Task 1.6
Now, repeat the byte pair encoding for vocab sizes of 1000 and 5000. What do you notice?

Our observation reveals that increasing the vocabulary size leads to an improved learning of the tokenizer, as it gains access to a greater number of BPE rules. Our findings confirm this statement, as the total length of the sentence after tokenization is reduced when utilizing a larger vocabulary and a greater number of BPE rules. Please refer to the provided prints for the respective lengths at the end of this block.

In [ ]:
# Use this chunk for the answer
vocab_size = 1000
merges = {("i", "n"): "in"}
vocab.append("in")
while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits_1000)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits_1000 = merge_pair(*best_pair, splits_1000)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

In [ ]:
print(tokenize(sentence_example))
print(f"Length with vocab size of 1000 == {len(tokenize(sentence_example))}")

['D', 'ar', 'm', 'st', 'ad', 't', 'Ġh', 'old', 's', 'Ġth', 'e', 'Ġoffic', 'ial', 'Ġ', 'tit', 'le', 'ĠC', 'ity', 'Ġo', 'f', 'ĠS', 'ci', 'ence', 'Ġ', '(', 'G', 'er', 'man', ':', 'ĠW', 'iss', 'ens', 'ch', 'a', 'f', 'ts', 'st', 'ad', 't', ')', 'Ġas', 'Ġ', 'it', 'Ġ', 'is', 'Ġa', 'Ġma', 'j', 'or', 'Ġcent', 're', 'Ġo', 'f', 'Ġs', 'ci', 'enti', 'f', 'ic', 'Ġin', 's', 'tit', 'u', 'tions', ',', 'Ġun', 'iv', 'ers', 'ities', ',', 'Ġ', 'and', 'Ġhigh', '-', 't', 'ech', 'n', 'ology', 'Ġcomp', 'an', 'ies', '.']
Length with vocab size of 1000 == 81


In [ ]:
# Use this chunk for the answer
vocab_size = 5000
merges = {("i", "n"): "in"}
vocab.append("in")
while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits_5000)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits_5000 = merge_pair(*best_pair, splits_5000)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

In [ ]:
print(tokenize(sentence_example))
print(f"Length with vocab size of 5000 == {len(tokenize(sentence_example))}")

['D', 'ar', 'm', 'st', 'ad', 't', 'Ġhold', 's', 'Ġthe', 'Ġofficial', 'Ġtitle', 'ĠC', 'ity', 'Ġof', 'ĠSci', 'ence', 'Ġ(', 'G', 'er', 'man', ':', 'ĠW', 'iss', 'ens', 'ch', 'a', 'f', 'ts', 'st', 'ad', 't', ')', 'Ġas', 'Ġit', 'Ġis', 'Ġa', 'Ġmajor', 'Ġcent', 're', 'Ġof', 'Ġs', 'ci', 'entific', 'Ġins', 'tit', 'u', 'tions', ',', 'Ġun', 'iv', 'ers', 'ities', ',', 'Ġand', 'Ġhigh', '-', 't', 'ech', 'n', 'ology', 'Ġcomp', 'an', 'ies', '.']
Length with vocab size of 5000 == 64


# **Task 2: Getting to Know Foundation Models**

In this task, we want to load two different versions of GPT-2 and compare their performance.

### Task 2.1

After we imported transformers from Huggingface, your first task is to initialize the GPT2LMHead Model. This model should be a gpt2 model from huggingface under the variable model. Also define the GPT2 tokenizer.

While we have seen the transformers in the lectures, you should print a single GPT2Block to the console for the 10th layer. Then, count the parameters of the model.

In [ ]:
# initialize model and tokenizer
model_small = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer_gpt2 = GPT2Tokenizer.from_pretrained("gpt2")

In [ ]:
# Print a GPT2Block in the 10th layer
# There are 12 layers, the index begins at 0, there the 10th layer has an index of 9
print(f"10th layer of GPT2LMHead: \n{model_small.transformer.h[9]}")

def count_parameters(model):
    #YOUR CODE HERE
    #trainable are equal to total in this case
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# GPT2_small parameter trainable parameters:
print(f"The model has {count_parameters(model_small):,} trainable parameters")

10th layer of GPT2LMHead: 
GPT2Block(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2Attention(
    (c_attn): Conv1D()
    (c_proj): Conv1D()
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D()
    (c_proj): Conv1D()
    (act): NewGELUActivation()
    (dropout): Dropout(p=0.1, inplace=False)
  )
)
The model has 124,439,808 trainable parameters


### Task 2.2
Because we want to compare two versions of GPT, you now also load the gpt2-large from huggingface as model. As evaluation dataset you import the dataset "wikitext-2-raw-v1" from huggingface. (like in Task1) Make sure to use the 200 first entries of the test dataset and print an entry. Also print the parameters of the gpt2-large model

In [ ]:
# Load the gpt2-large model from huggingface
model_large = GPT2LMHeadModel.from_pretrained("gpt2-large")
tokenizer_gpt2_large = GPT2Tokenizer.from_pretrained("gpt2-large")

In [ ]:
# Load the dataset
whole_dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
test_dataset = whole_dataset["test"]["text"][:200]
# Print any single entry
print(f'Second entry: {test_dataset[1]}')
# Print Trainable parameters of huggingface gpt2-large
print(f"The GPT2-LARGE model has {count_parameters(model_large):,} trainable parameters")

  0%|          | 0/3 [00:00<?, ?it/s]

Second entry:  = Robert Boulter = 

The GPT2-LARGE model has 774,030,080 trainable parameters


### Task 2.3
As a next step, we define an evaluation metric to calculate predictions. Remember the perplexity funciton from the lecture. The function should take a model, tokenizer and a text and return the perplexity score for the given text.

Hint: you should encode the text, generate outputs for inputs and corresponding labels and then calculate perplexity.

In [ ]:
# useful article: https://medium.com/@priyankads/perplexity-of-language-models-41160427ed72
def calculate_perplexity(model, tokenizer, text):
    # YOUR CODE HERE
    inputs = tokenizer(text, return_tensors="pt")
    loss = model(input_ids=inputs["input_ids"], labels=inputs["input_ids"]).loss
    ppl = torch.exp(loss)
    #only return the loss value without the backpropagation item
    return ppl.item()

In [ ]:
import numpy as np
from tqdm import tqdm

perplexities_small = []
perplexities_large = []

# Loop over the dataset, append scores to the lists
for idx, sent in enumerate(tqdm(test_dataset, desc="Processing")):
  try:
    perplexities_large.append(calculate_perplexity(model=model_large, tokenizer=tokenizer_gpt2_large, text=sent))
  except Exception as e:
    perplexities_large.append(np.nan)

  try:
    perplexities_small.append(calculate_perplexity(model=model_small, tokenizer=tokenizer_gpt2, text=sent))
  except Exception as e:
    perplexities_small.append(np.nan)

# Calculate and print average perplexities
print("\nAverage perplexity (GPT2): ", np.nanmean(perplexities_small))
print("Average perplexity (GPT2 large): ", np.nanmean(perplexities_large))

Processing: 100%|██████████| 200/200 [10:12<00:00,  3.06s/it]


Average perplexity (GPT2):  509.78572745477
Average perplexity (GPT2 large):  433.0942020723897


What do you notice? Please explain.

We notice that the GPT2 large model has a lower perplexity value than the small model.

The lower perplexity value of the GPT2 large model suggests that it has a better ability to predict the next word in a given sequence compared to the GPT2 small model. This improvement can be attributed to the larger size or more advanced architecture of the GPT2 large model due to the bigger number of trainable parameters, which enables it to capture more intricate language patterns and dependencies.

In summary, based on the perplexity results, the GPT2 large model demonstrates better performance in terms of language modeling compared to the GPT2 small model.

### Task 2.4 Masking attention
What is an attention mask? When and why is it usually used?

Your answer here:

An attention mask is a binary mask (list of 0s and 1s) used in attention mechanisms, particularly in transformer-based models, to indicate which tokens in the input sequence should be taken into consideration (valid tokens) and which should be ignored (padding tokens).

By using attention masks, the model can effectively handle sequences of different lengths while maintaining accurate attention and avoiding interference from irrelevant tokens.


Your task is now to run a prompt in regular inference, but with an attention mask. You can reuse the gpt2_small model

In [ ]:
text_prompt = "The sun was setting behind"

# Step 1: Implement attention masking
def apply_attention_mask(input_ids, mask_start_idx, mask_end_idx):
    # -----------------------
    #YOUR CODE HERE
    # -----------------------
    attention_mask = torch.ones(input_ids.shape)
    # mask the given positions
    attention_mask[:, mask_start_idx : mask_end_idx + 1] = 0
    return attention_mask

# Step 2: Apply attention masks
mask_start = 3  # Start index of the region to mask
mask_end = 5  # End index of the region to mask
tokenizer_small = AutoTokenizer.from_pretrained("gpt2")
input_ids = tokenizer_small.encode(text_prompt, return_tensors='pt')
attention_mask = apply_attention_mask(input_ids, mask_start, mask_end)

# Step 3: Generate output using modified attention mechanism
with torch.no_grad():
    outputs = model_small.generate(input_ids, attention_mask=attention_mask)
    generated_text = tokenizer_small.decode(outputs[0], skip_special_tokens=True)

# Step 4: Compare and analyze the generated outputs
print("\nText Prompt:", text_prompt)
print("Generated Text:", generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Text Prompt: The sun was setting behind
Generated Text: The sun was setting behind the only thing that could stop him.

"I'm sorry,


In [ ]:
# Step 5: Run the same prompt without attention mask

# YOUR CODE HERE
with torch.no_grad():
    outputs = model_small.generate(input_ids)
    generated_text = tokenizer_small.decode(outputs[0], skip_special_tokens=True)

# Step 4: Compare and analyze the generated outputs
print("\nText Prompt:", text_prompt)
print("Generated Text NO ATTENTION MASK:", generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Text Prompt: The sun was setting behind
Generated Text NO ATTENTION MASK: The sun was setting behind the clouds, and the wind was blowing in the direction of the sun.


Step 6: Compare the two outputs and describe it in one to two sentences

Your answer here:

The output of the model without applying the attention mask is more suitable for our input because it uses the entire sentence as context, whereas applying this specific attention mask would take off a lot of the context of the sentence which makes it harder for the model to predict the next words.